To reproduce the final score, run this notebook as it is. We tried various techniques, have left them here for reference. We have commented the steps that did not boost the score.

# **Imports**

In [ ]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd
from scipy import sparse

import matplotlib.pyplot as plt
import seaborn as sns

# **Mounting Gogole Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Import data from drive (kindly edit this path)**

In [ ]:
# convert csv to dataframe
x = pd.read_csv('/content/drive/MyDrive/Data/ML_567_Project_Data/train_values.csv',index_col='building_id')
y = pd.read_csv('/content/drive/MyDrive/Data/ML_567_Project_Data/train_labels.csv',index_col='building_id')
x_comp_test = pd.read_csv('/content/drive/MyDrive/Data/ML_567_Project_Data/test_values.csv',index_col='building_id')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV

from timeit import default_timer as timer

!pip install catboost
!pip install optuna

from catboost import CatBoostClassifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
x_subfeatures =x
x_comp_test = x_comp_test

In [ ]:
display(x_subfeatures.head())

geo_level_1_id  geo_level_2_id  geo_level_3_id  \
building_id                                                   
802906                    6             487           12198   
28830                     8             900            2812   
94947                    21             363            8973   
590882                   22             418           10694   
201944                   11             131            1488   

             count_floors_pre_eq  age  area_percentage  height_percentage  \
building_id                                                                 
802906                         2   30                6                  5   
28830                          2   10                8                  7   
94947                          2   10                5                  5   
590882                         2   10                6                  5   
201944                         3   30                8                  9   

            land_surface_condition foundation_type roof_type  ...  \
building_id                                                   ...   
802906                           t               r         n  ...   
28830                            o               r         n  ...   
94947                            t               r         n  ...   
590882                           t               r         n  ...   
201944                           t               r         n  ...   

            has_secondary_use_agriculture has_secondary_use_hotel  \
building_id                                                         
802906                                  0                       0   
28830                                   0                       0   
94947                                   0                       0   
590882                                  0                       0   
201944                                  0                       0   

            has_secondary_use_rental has_secondary_use_institution  \
building_id                                                          
802906                             0                             0   
28830                              0                             0   
94947                              0                             0   
590882                             0                             0   
201944                             0                             0   

             has_secondary_use_school  has_secondary_use_industry  \
building_id                                                         
802906                              0                           0   
28830                               0                           0   
94947                               0                           0   
590882                              0                           0   
201944                              0                           0   

             has_secondary_use_health_post  has_secondary_use_gov_office  \
building_id                                                                
802906                                   0                             0   
28830                                    0                             0   
94947                                    0                             0   
590882                                   0                             0   
201944                                   0                             0   

             has_secondary_use_use_police  has_secondary_use_other  
building_id                                                         
802906                                  0                        0  
28830                                   0                        0  
94947                                   0                        0  
590882                                  0                        0  
201944                                  0                        0  

[5 rows x 38 columns]

In [ ]:

x_train_subset=x_subfeatures
y_train_subset = y

In [ ]:
x_train_subset.pivot_table(index=y_train_subset['damage_grade'], aggfunc='size').plot(kind='bar', title='Verify that class distributuion in train is same as input data')

<Axes: title={'center': 'Verify that class distributuion in train is same as input data'}, xlabel='damage_grade'>

# **Combining geo features**

In [ ]:
#Combining geo features
# x_train_subset['geo_feature_combined'] = x_train_subset['geo_level_1_id'].astype(str)+ "."+x_train_subset['geo_level_2_id'].astype(str)+ "."+x_train_subset['geo_level_3_id'].astype(str)
# x_train_subset.drop(columns=['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'], axis=1)

# x_comp_test['geo_feature_combined'] = x_comp_test['geo_level_1_id'].astype(str)+ "."+x_comp_test['geo_level_2_id'].astype(str)+ "."+x_comp_test['geo_level_3_id'].astype(str)
# x_comp_test.drop(columns=['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'], axis=1)

In [ ]:
# x_train_subset['geo_feature_combined'].head()

# **Correlation plot**

In [ ]:
# plt.figure(figsize=(15,8))
# sns.countplot(x=x_train_subset['geo_level_2_id'],hue=x_train_subset["damage_grade"],palette="viridis")
# plt.ylabel("Damage Grade")
# plt.title("Geo feature")
# plt.legend(["Low damage","Avg damage","High damage"],loc="upper right")
# plt.xticks(rotation=45)
# plt.show()

# **Label Encoding**

In [ ]:
#Label Encoding
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

x_train_subset['land_surface_condition']= label_encoder.fit_transform(x_train_subset['land_surface_condition'])
x_train_subset['foundation_type']= label_encoder.fit_transform(x_train_subset['foundation_type'])
x_train_subset['roof_type']= label_encoder.fit_transform(x_train_subset['roof_type'])
x_train_subset['ground_floor_type']= label_encoder.fit_transform(x_train_subset['ground_floor_type'])
x_train_subset['other_floor_type']= label_encoder.fit_transform(x_train_subset['other_floor_type'])
x_train_subset['position']= label_encoder.fit_transform(x_train_subset['position'])
x_train_subset['plan_configuration']= label_encoder.fit_transform(x_train_subset['plan_configuration'])
x_train_subset['legal_ownership_status']= label_encoder.fit_transform(x_train_subset['legal_ownership_status'])

x_comp_test['land_surface_condition']= label_encoder.fit_transform(x_comp_test['land_surface_condition'])
x_comp_test['foundation_type']= label_encoder.fit_transform(x_comp_test['foundation_type'])
x_comp_test['roof_type']= label_encoder.fit_transform(x_comp_test['roof_type'])
x_comp_test['ground_floor_type']= label_encoder.fit_transform(x_comp_test['ground_floor_type'])
x_comp_test['other_floor_type']= label_encoder.fit_transform(x_comp_test['other_floor_type'])
x_comp_test['position']= label_encoder.fit_transform(x_comp_test['position'])
x_comp_test['plan_configuration']= label_encoder.fit_transform(x_comp_test['plan_configuration'])
x_comp_test['legal_ownership_status']= label_encoder.fit_transform(x_comp_test['legal_ownership_status'])

# **Categorical feature tagging**

In [ ]:
# all_cat_features = categorical + binary + categorical_many
all_cat_features= ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']

In [ ]:
print("# Train Values: {}".format(x_train_subset.shape))
print("# Train Labels: {}".format(y_train_subset.shape))
print("# Test Values: {}".format(x_comp_test.shape))

# Train Values: (260601, 38)
# Train Labels: (260601, 1)
# Test Values: (86868, 38)


In [ ]:
X = x_train_subset
y = y_train_subset

# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.2, random_state = 42)
X_train = X
y_train = y

# **Target encoding**

In [ ]:
# !pip install category-encoders
# from category_encoders import TargetEncoder
# targets = y_train['damage_grade'].unique()
# for t in targets:
#     target_aux = y_test['damage_grade'].apply(lambda x: 1 if x == t else 0)
#     encoder = TargetEncoder()
#     X_test['geo_level_1_id_' + str(t)] = encoder.fit_transform(X_test['geo_level_1_id'], target_aux)
#     X_train['geo_level_1_id_' + str(t)] = encoder.transform(X_train['geo_level_1_id'])
#     x_comp_test['geo_level_1_id_' + str(t)] = encoder.transform(x_comp_test['geo_level_1_id'])
#     x['geo_level_1_id_' + str(t)] = encoder.transform(x['geo_level_1_id'])

# for t in targets:
#     target_aux = y_test['damage_grade'].apply(lambda x: 1 if x == t else 0)
#     encoder = TargetEncoder()
#     X_test['geo_level_2_id_' + str(t)] = encoder.fit_transform(X_test['geo_level_2_id'], target_aux)
#     X_train['geo_level_2_id_' + str(t)] = encoder.transform(X_train['geo_level_2_id'])
#     x_comp_test['geo_level_2_id_' + str(t)] = encoder.transform(x_comp_test['geo_level_2_id'])
#     x['geo_level_2_id_' + str(t)] = encoder.transform(x['geo_level_2_id'])
  
# for t in targets:
#     target_aux = y_test['damage_grade'].apply(lambda x: 1 if x == t else 0)
#     encoder = TargetEncoder()
#     X_test['geo_level_3_id_' + str(t)] = encoder.fit_transform(X_test['geo_level_3_id'], target_aux)
#     X_train['geo_level_3_id_' + str(t)] = encoder.transform(X_train['geo_level_3_id'])
#     x_comp_test['geo_level_3_id_' + str(t)] = encoder.transform(x_comp_test['geo_level_3_id'])
#     x['geo_level_3_id_' + str(t)] = encoder.transform(x['geo_level_3_id'])
    

Grid Search for optimal parameter tuning

In [ ]:
# param_dist = {
#               "max_depth": np.linspace(75,125,5, dtype=np.int64),
#               "max_leaf_nodes": np.linspace(2000,2500,10, dtype=np.int64),
    
# }
# tree_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), param_dist, cv=5, n_jobs=-1, verbose=10)
# tree_cv.fit(x_train,y)
# print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
# print("Best score is {}".format(tree_cv.best_score_))

In [ ]:
# X_test.head()

# **CatBoost model**

In [ ]:
cat_model = CatBoostClassifier( cat_features = all_cat_features,loss_function='MultiClass', task_type="CPU",
                               iterations=3000, max_depth=9, eval_metric='TotalF1', verbose=True, l2_leaf_reg=3, learning_rate=0.05)

In [ ]:
# result =cat_model.fit(X_train,y_train.values.ravel(), early_stopping_rounds= 100)

# preds = result.predict(X_test)

In [ ]:
# from sklearn.metrics import f1_score
# print("F1 SCORE: {}".format(f1_score(y_test, preds, average='micro')))

# **Finding feature importances**

In [ ]:
# df_importances=pd.DataFrame(cat_model.feature_importances_)

# df_importances.index=x_train_subset.columns
# df_importances.columns=['importances']
# display(df_importances.sort_values(by='importances', ascending=False).head(25).plot.bar())
# df_importances.to_csv( 'CAT_importances.csv')

In [ ]:
# from sklearn.metrics import confusion_matrix
# data = confusion_matrix(y_test, preds)
# cm = pd.DataFrame(data, columns=np.unique(y_test), index = np.unique(y_test))
# cm.index.name = 'Actual'
# cm.columns.name = 'Predicted'
# plt.figure(figsize = (10,7))
# sns.heatmap(cm, annot=True, fmt="d", annot_kws={"size": 12})
# plt.title("Confusion Matrix")
# plt.show()

# **Fitting the model on training data**

In [ ]:
result =cat_model.fit(x,y.values.ravel(), early_stopping_rounds= 100)
submit_preds = result.predict(x_comp_test)

0:	learn: 0.6898885	total: 8s	remaining: 6h 40m 5s
1:	learn: 0.6891688	total: 20.1s	remaining: 8h 21m 15s
2:	learn: 0.6918082	total: 25.8s	remaining: 7h 9m 30s
3:	learn: 0.6909699	total: 30.2s	remaining: 6h 17m 31s
4:	learn: 0.6922490	total: 36s	remaining: 5h 59m 35s
5:	learn: 0.6921599	total: 39.8s	remaining: 5h 30m 58s
6:	learn: 0.6921425	total: 44.2s	remaining: 5h 14m 51s
7:	learn: 0.6928111	total: 50.5s	remaining: 5h 14m 29s
8:	learn: 0.6927019	total: 54.1s	remaining: 4h 59m 46s
9:	learn: 0.6927176	total: 58.1s	remaining: 4h 49m 17s
10:	learn: 0.6927176	total: 1m 4s	remaining: 4h 52m 35s
11:	learn: 0.6928615	total: 1m 8s	remaining: 4h 43m 55s
12:	learn: 0.6929554	total: 1m 12s	remaining: 4h 37m 30s
13:	learn: 0.6928553	total: 1m 16s	remaining: 4h 33m 36s
14:	learn: 0.6935803	total: 1m 21s	remaining: 4h 31m 14s
15:	learn: 0.6935473	total: 1m 25s	remaining: 4h 25m 40s
16:	learn: 0.6934496	total: 1m 29s	remaining: 4h 21m 10s
17:	learn: 0.6937969	total: 1m 35s	remaining: 4h 24m 43s
18:

# **Predictions on test data**

In [ ]:
submission_format = pd.read_csv('/content/drive/MyDrive/Data/ML_567_Project_Data/submission_format.csv', index_col='building_id')
my_submission = pd.DataFrame(data=submit_preds,
                             columns=submission_format.columns,
                             index=submission_format.index)

my_submission.to_csv('submission.csv')
!head submission.csv

building_id,damage_grade
300051,3
99355,2
890251,2
745817,1
421793,3
871976,2
691228,2
896100,3
343471,2
